# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [3]:
import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import ttest_1samp
from statsmodels.stats import weightstats
from scipy.stats import ttest_rel   # con muestras dependientes
from scipy.stats import ttest_ind   # con muestras independientes
from scipy.stats import shapiro
from sklearn.preprocessing import StandardScaler
from scipy.stats import sem   # error standard
from statsmodels.stats.proportion import proportions_ztest


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [4]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [5]:
# Your code here:

salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [7]:
# Your code here:

salaries.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [8]:
# Your code here:

salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [14]:
# Your code here:

dep_counts = salaries.groupby('Department')['Name'].count()


print(dep_counts)

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [28]:
# Your code here:
Hourly = salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate']

t_test = ttest_1samp(Hourly, 30, alternative='two-sided')

t_test.pvalue


4.3230240486229894e-92

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [37]:
# Your code here:

# H0 = los sueldos son menores que 86000

# H! = los sueldos son mayores de 86000
mu = 86000

PD = salaries.loc[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE'), 'Annual Salary']

t_test = ttest_1samp(PD, mu, alternative='greater')

t_test.pvalue




0.0010301701775482569

In [45]:
t_test

### Puedo rechazar la H0 

TtestResult(statistic=3.081997005712994, pvalue=0.0010301701775482569, df=13403)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [52]:
# Your code here:
HW = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'] == 'Hourly')

print(HW)

HW.idxmax()

Salary or Hourly       False  True 
Department                         
ADMIN HEARNG              39      0
ANIMAL CONTRL             62     19
AVIATION                 547   1082
BOARD OF ELECTION        107      0
BOARD OF ETHICS            8      0
BUDGET & MGMT             44      2
BUILDINGS                269      0
BUSINESS AFFAIRS         164      7
CITY CLERK                84      0
CITY COUNCIL             347     64
COMMUNITY DEVELOPMENT    203      4
COPA                     116      0
CULTURAL AFFAIRS          58      7
DISABILITIES              28      0
DoIT                      99      0
FAMILY & SUPPORT         328    287
FINANCE                  516     44
FIRE                    4639      2
GENERAL SERVICES         215    765
HEALTH                   485      3
HUMAN RELATIONS           16      0
HUMAN RESOURCES           75      4
INSPECTOR GEN             87      0
LAW                      367     40
LICENSE APPL COMM          1      0
MAYOR'S OFFICE            77

Salary or Hourly
False           POLICE
True     STREETS & SAN
dtype: object

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [54]:
# H0 = mayor a 35
# H1 = menor a 35

mu=35

SS = salaries.loc[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN'), 'Hourly Rate']

t_test = ttest_1samp(SS, mu, alternative='greater')

t_test.pvalue


1.0

In [55]:
# No hay suficiente evidencia para rechazar la H1

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [63]:
# Your code here:
from scipy.stats import sem
from scipy.stats import t

dfn = salaries['Hourly Rate'].dropna()

interval = t.interval(0.95,
                           len(salaries['Salary or Hourly'] == 'Hourly') -1,
                            loc= dfn.mean (),
                            scale=sem(dfn))

interval


(32.52348867934135, 33.05362675321913)

In [64]:
dfn.mean()

32.78855771628024

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [66]:
# Your code here:

df4 = salaries[salaries['Department'] == 'POLICE']

df4 = df4['Annual Salary'].dropna()



interval = t.interval(0.95,
                           len(salaries['Salary or Hourly'] == 'Salary') -1,
                            loc= df4.mean (),
                            scale=sem(df4))

interval




(86177.07296831945, 86795.75603794733)

In [67]:
df4.mean()

86486.41450313339

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [70]:
# Your code here:

z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()[1],     # 25 elementos de a, 
                                    len(salaries),  # numero total de elementos
                                    0.25           # proporcion
                                   ) 

z_score, p_value

(-3.5099964213703005, 0.0004481127249057967)

In [71]:
salaries['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

In [72]:
z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()[1],     # 25 elementos de a, 
                                    len(salaries),  # numero total de elementos
                                    0.95           # proporcion
                                   ) 

z_score, p_value

(-301.3384818103434, 0.0)